In [8]:
import time
import os
import csv
from dotenv import load_dotenv
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

load_dotenv()
FACEBOOK_EMAIL = os.getenv('FACEBOOK_EMAIL')
FACEBOOK_PASSWORD = os.getenv('FACEBOOK_PASSWORD')
PAGE_URL = 'https://m.facebook.com/groups/322977734828852'
OUTPUT_CSV_FILE = 'Facebook_group_post_urls_last5y.csv'
PROFILE_PATH = r'C:\chrome-profiles\fb-pipeline-stage1-persistent'

def login_to_facebook(driver):
    driver.get("https://m.facebook.com")
    time.sleep(3)
    selectors = [
        "button[data-cookiebanner='accept_button_dialog']",
        "button[title='Allow all cookies']",
        "button[title='Accept All']",
        "button[aria-label='Allow all cookies']"
    ]
    for s in selectors:
        btns = driver.find_elements(By.CSS_SELECTOR, s)
        if btns and btns[0].is_displayed():
            btns[0].click()
            time.sleep(2)
            break
    email = driver.find_elements(By.ID, "m_login_email")
    pwd = driver.find_elements(By.ID, "m_login_password")
    if not email:
        email = driver.find_elements(By.ID, "email")
        pwd = driver.find_elements(By.ID, "pass")
    if email and pwd:
        email[0].clear()
        email[0].send_keys(FACEBOOK_EMAIL)
        pwd[0].send_keys(FACEBOOK_PASSWORD)
        pwd[0].send_keys(Keys.RETURN)
        WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[aria-label='Menu'], a[aria-label='Home'], a[aria-label='Search']")))

def collect_group_posts_last_5y(driver, group_url):
    cutoff_epoch = int(time.time() - 5 * 365 * 24 * 60 * 60)
    driver.get(group_url)
    WebDriverWait(driver, 30).until(EC.presence_of_element_located((By.CSS_SELECTOR, "div[role='main'], #m_group_stories_container, article")))
    time.sleep(3)
    group_id = group_url.rstrip('/').split('/')[-1]
    urls = set()
    prev_len = 0
    stagnant = 0
    reached_old = False
    loops = 0
    while True:
        data = driver.execute_script("""
            var gid = arguments[0];
            var anchors = Array.from(document.querySelectorAll("a[href*='/groups/']"));
            var out = [];
            for (var i=0;i<anchors.length;i++){
                var a = anchors[i];
                var href = a.getAttribute("href") || "";
                if(!href) continue;
                if(href.indexOf("/groups/"+gid+"/") === -1 && href.indexOf("story.php") === -1) continue;
                if(href.indexOf("/reel") !== -1 || href.indexOf("/reels/") !== -1 || href.indexOf("/videos/") !== -1 || href.indexOf("/watch/") !== -1 || href.indexOf("video_id=") !== -1) continue;
                if(href.indexOf("/permalink/") === -1 && href.indexOf("/posts/") === -1 && href.indexOf("story.php") === -1) continue;
                if(href.indexOf("http")!==0){ href = location.origin + href; }
                href = href.split("?")[0];
                var el = a.closest("article") || a.closest("div");
                var ut = null;
                if(el){
                    var t = el.querySelector("abbr[data-utime], time[data-utime]");
                    if(t){ ut = parseInt(t.getAttribute("data-utime")); }
                    if(!ut){
                        var t2 = el.querySelector("time[datetime], abbr[datetime]");
                        if(t2 && t2.getAttribute("datetime")){
                            var ms = Date.parse(t2.getAttribute("datetime"));
                            if(!isNaN(ms)) ut = Math.floor(ms/1000);
                        }
                    }
                }
                out.push([href, ut]);
            }
            return out;
        """, group_id)
        for href, ut in data:
            if ut is None or ut >= cutoff_epoch:
                urls.add(href)
        if any((ut is not None and ut < cutoff_epoch) for _, ut in data):
            reached_old = True
        see_more = driver.find_elements(By.XPATH, "//div[@role='button' and (contains(., 'See more') or contains(., 'See More'))]")
        if see_more:
            see_more[0].click()
            time.sleep(1.2)
        driver.execute_script("window.scrollBy(0, arguments[0]);", 1200)
        time.sleep(2.0)
        curr_len = len(urls)
        if curr_len == prev_len:
            stagnant += 1
        else:
            stagnant = 0
        prev_len = curr_len
        loops += 1
        if (reached_old and stagnant >= 2) or stagnant >= 6 or loops >= 3000:
            break
    return urls

def main():
    options = uc.ChromeOptions()
    options.add_argument("--disable-notifications")
    options.add_argument("--lang=en-US")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--no-sandbox")
    options.add_argument(f"--user-data-dir={PROFILE_PATH}")
    options.page_load_strategy = "eager"
    driver = uc.Chrome(options=options, use_subprocess=True)
    driver.set_page_load_timeout(90)
    driver.set_script_timeout(90)
    driver.command_executor._client_config.timeout = 300
    login_to_facebook(driver)
    all_urls = collect_group_posts_last_5y(driver, PAGE_URL)
    driver.quit()
    if all_urls:
        with open(OUTPUT_CSV_FILE, 'w', newline='', encoding='utf-8') as f:
            w = csv.writer(f)
            w.writerow(['PostURL'])
            for u in sorted(all_urls):
                w.writerow([u])

if __name__ == "__main__":
    main()

TimeoutException: Message: timeout: Timed out receiving message from renderer: 90.000
  (Session info: chrome=140.0.7339.129)
Stacktrace:
	GetHandleVerifier [0x0xa7c333+65459]
	GetHandleVerifier [0x0xa7c374+65524]
	(No symbol) [0x0x89d973]
	(No symbol) [0x0x88e184]
	(No symbol) [0x0x88deba]
	(No symbol) [0x0x88bdc0]
	(No symbol) [0x0x88c8a7]
	(No symbol) [0x0x8994fe]
	(No symbol) [0x0x8aaec5]
	(No symbol) [0x0x8b0a56]
	(No symbol) [0x0x88cf26]
	(No symbol) [0x0x8aab6e]
	(No symbol) [0x0x8aa921]
	(No symbol) [0x0x92b515]
	(No symbol) [0x0x909bf6]
	(No symbol) [0x0x8db38e]
	(No symbol) [0x0x8dc274]
	GetHandleVerifier [0x0xcfeda3+2697763]
	GetHandleVerifier [0x0xcf9ec7+2677575]
	GetHandleVerifier [0x0xaa4194+228884]
	GetHandleVerifier [0x0xa949f8+165496]
	GetHandleVerifier [0x0xa9b18d+192013]
	GetHandleVerifier [0x0xa847d8+99416]
	GetHandleVerifier [0x0xa84972+99826]
	GetHandleVerifier [0x0xa6ebea+10346]
	BaseThreadInitThunk [0x0x765d5d49+25]
	RtlInitializeExceptionChain [0x0x772cd6db+107

In [6]:
import time
import os
import csv
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from dotenv import load_dotenv

load_dotenv()
FACEBOOK_EMAIL = os.getenv('FACEBOOK_EMAIL')
FACEBOOK_PASSWORD = os.getenv('FACEBOOK_PASSWORD')
INPUT_LINKS_CSV = 'Facebook_group_post_urls_last5y.csv'
OUTPUT_DETAILS_CSV = 'scraped_full_content.csv'
PROFILE_PATH = r'C:\\chrome-profiles\\fb-scraper-profile'
WEBDRIVER_WAIT_TIMEOUT = 20

def get_raw_post_text(driver, permalink):
    wait = WebDriverWait(driver, WEBDRIVER_WAIT_TIMEOUT)
    driver.get(permalink)

    dialog_selector = "div[role='dialog']"
    dialog_element = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, dialog_selector)))

    see_more_xpath = ".//div[@role='button' and (contains(., 'See more') or contains(., 'เพิ่มเติม'))]"
    
    see_more_buttons = dialog_element.find_elements(By.XPATH, see_more_xpath)
    if see_more_buttons:
        driver.execute_script("arguments[0].click();", see_more_buttons[0])
        time.sleep(1.5)

    content_container_selector = 'div[data-ad-rendering-role="story_message"]'
    container = dialog_element.find_element(By.CSS_SELECTOR, content_container_selector)
    post_text = container.text
    return post_text

if not FACEBOOK_EMAIL or not FACEBOOK_PASSWORD:
    print("Error: Facebook credentials not found. Please check your .env file.")
    exit()

print("Initializing Chrome Driver...")
options = uc.ChromeOptions()
options.add_argument(f'--user-data-dir={PROFILE_PATH}')
options.add_argument('--disable-notifications')
driver = uc.Chrome(options=options)
wait = WebDriverWait(driver, WEBDRIVER_WAIT_TIMEOUT)

print("Logging into Facebook...")
driver.get("https://www.facebook.com")
time.sleep(3)

cookie_buttons = driver.find_elements(By.CSS_SELECTOR, 'button[data-cookiebanner="accept_button"]')
if cookie_buttons and cookie_buttons[0].is_displayed():
    cookie_buttons[0].click()
    time.sleep(1)

if "login" in driver.current_url:
    wait.until(EC.presence_of_element_located((By.ID, "email"))).send_keys(FACEBOOK_EMAIL)
    driver.find_element(By.ID, "pass").send_keys(FACEBOOK_PASSWORD)
    driver.find_element(By.NAME, "login").click()

wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'div[role="main"]')))
print("Login successful. Starting scraping process...")

with open(INPUT_LINKS_CSV, 'r', encoding='utf-8') as infile, \
      open(OUTPUT_DETAILS_CSV, 'w', newline='', encoding='utf-8') as outfile:

    reader = csv.reader(infile)
    writer = csv.writer(outfile)

    header = ['Post_URL', 'Full_Post_Content']
    writer.writerow(header)

    next(reader, None)

    for i, row in enumerate(reader):
        if not row:
            continue
        permalink = row[0]
        if not permalink:
            continue

        print(f"Processing link {i+1}: {permalink}")
        
        raw_text = "N/A"
        try:
            raw_text = get_raw_post_text(driver, permalink)
        except Exception as e:
            print(f"  - FAILED to process link. Reason: {e}")
            raw_text = 'ERROR: Main process failed.'

        output_row = [permalink, raw_text]
        writer.writerow(output_row)

        time.sleep(3)

print(f"Scraping complete. Data saved to {OUTPUT_DETAILS_CSV}")
driver.quit()

Initializing Chrome Driver...
Logging into Facebook...
Login successful. Starting scraping process...
Processing link 1: https://www.facebook.com/groups/322977734828852/posts/1725715287888416/
Processing link 2: https://www.facebook.com/groups/322977734828852/posts/1855015104958433/
Processing link 3: https://www.facebook.com/groups/322977734828852/posts/1886307071829236/
Processing link 4: https://www.facebook.com/groups/322977734828852/posts/1976119702847972/
Processing link 5: https://www.facebook.com/groups/322977734828852/posts/2028623960930879/
Processing link 6: https://www.facebook.com/groups/322977734828852/posts/2116491898810751/
Processing link 7: https://www.facebook.com/groups/322977734828852/posts/2120534288406512/
Processing link 8: https://www.facebook.com/groups/322977734828852/posts/2154369618356312/
Processing link 9: https://www.facebook.com/groups/322977734828852/posts/2173414889785118/
Processing link 10: https://www.facebook.com/groups/322977734828852/posts/22072

In [ ]:
import csv
import re

INPUT_CSV = 'scraped_full_content.csv'
OUTPUT_CSV = 'extracted_property_details.csv'

def extract_details(text):
    prop_type = 'N/A'
    property_types = ['บ้านเดี่ยว', 'ทาวน์โฮม', 'ทาวน์เฮ้าส์', 'คอนโด', 'ที่ดิน', 'อาคารพาณิชย์', 'ตึกแถว', 'สวนลำไย']
    for p_type in property_types:
        if p_type in text:
            prop_type = p_type
            break
    if 'บ้านพร้อมที่ดิน' in text:
        prop_type = 'บ้านพร้อมที่ดิน'

    price = 'N/A'
    price_match = re.search(r'([\d,.]+)\s*(ล้าน|บาท|/ด)', text)
    if price_match:
        price = price_match.group(0).strip()
    price_match_2 = re.search(r'(\d{4,})/ด', text)
    if not price_match and price_match_2:
        price = price_match_2.group(0).strip()

    location = 'N/A'
    location_keywords = [
        'หมู่บ้าน', 'โครงการ', 'ถนน', 'ซอย', 'ตำบล', 'อำเภอ', 'ย่าน', 'ติด', 'ใกล้',
        'สันกำแพง', 'สันทราย', 'หางดง', 'สารภี', 'แม่ริม', 'รวมโชค', 'หนองจ๊อม',
        'แม่โจ้', 'จอมทอง', 'ป่าซาง'
    ]
    loc_pattern = r'(' + '|'.join(location_keywords) + r')[\s\w.()-]+'
    loc_match = re.search(loc_pattern, text)
    if loc_match:
        location = loc_match.group(0).strip().replace('\n', ' ')

    size_parts = []
    land_size_matches = re.findall(r'(\d[\d,.]*\s*(?:ไร่|งาน|ตร\.?ว\.?|ตารางวา))', text)
    if land_size_matches:
        size_parts.extend(land_size_matches)
    beds_match = re.search(r'(\d+)\s*(?:ห้อง)?นอน', text)
    if beds_match:
        size_parts.append(f"{beds_match.group(1)} นอน")
    baths_match = re.search(r'(\d+)\s*(?:ห้อง)?น้ำ', text)
    if baths_match:
        size_parts.append(f"{baths_match.group(1)} น้ำ")
        
    size = ' '.join(size_parts) if size_parts else 'N/A'

    return {
        'ประเภท': prop_type,
        'ราคา': price,
        'ทำเล': location,
        'ขนาด': size,
    }

if __name__ == "__main__":
    with open(INPUT_CSV, mode='r', encoding='utf-8') as infile, \
         open(OUTPUT_CSV, mode='w', encoding='utf-8', newline='') as outfile:
        
        reader = csv.DictReader(infile)
        
        fieldnames = ['ประเภท', 'ราคา', 'ทำเล', 'ขนาด', 'Link']
        writer = csv.DictWriter(outfile, fieldnames=fieldnames)
        writer.writeheader()

        for row in reader:
            post_url = row.get('Post_URL', '')
            full_content = row.get('Full_Post_Content', '')

            details = extract_details(full_content)
            
            output_row = {
                'ประเภท': details['ประเภท'],
                'ราคา': details['ราคา'],
                'ทำเล': details['ทำเล'],
                'ขนาด': details['ขนาด'],
                'Link': post_url
            }
            writer.writerow(output_row)

In [ ]:
#DDproperty Scraper
import time
import csv
import re
from urllib.parse import urlparse, urlunparse, parse_qsl, urlencode
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

START_URL = 'https://www.ddproperty.com/%E0%B8%A3%E0%B8%A7%E0%B8%A1%E0%B8%9B%E0%B8%A3%E0%B8%B0%E0%B8%81%E0%B8%B2%E0%B8%A8%E0%B8%82%E0%B8%B2%E0%B8%A2?areaCode=&listingType=sale&regionCode=TH50&locale=th&slug=search&_freetextDisplay=%E0%B9%80%E0%B8%8A%E0%B8%B5%E0%B8%A2%E0%B8%87%E0%B9%83%E0%B8%AB%E0%B8%A1%E0%B9%88&isCommercial=false'
OUTPUT_CSV_FILE = 'ddproperty_listing_urls.csv'
WEBDRIVER_WAIT_TIMEOUT = 20
MAX_PAGES = 200

def build_page_url(start_url, page):
    u = urlparse(start_url)
    q = urlencode(dict(parse_qsl(u.query)), doseq=True)
    path = u.path.rstrip('/')
    m = re.match(r'^(.*?)(/\d+)$', path)
    if m:
        base = m.group(1)
    else:
        base = path
    if page <= 1:
        new_path = base
    else:
        new_path = base + f'/{page}'
    return urlunparse((u.scheme, u.netloc, new_path, u.params, q, u.fragment))

def main():
    print("Initializing Chrome Driver...")
    options = uc.ChromeOptions()
    options.add_argument('--disable-notifications')
    options.add_argument('--start-maximized')
    driver = uc.Chrome(options=options)
    wait = WebDriverWait(driver, WEBDRIVER_WAIT_TIMEOUT)

    first_url = build_page_url(START_URL, 1)
    print(f"Navigating to start URL: {first_url}")
    driver.get(first_url)

    btns = driver.find_elements(By.XPATH, "//button[normalize-space(text())='ยอมรับ']")
    if btns:
        btns[0].click()
        time.sleep(1)

    all_listing_urls = set()
    page_num = 1
    last_first_href = ""

    while page_num <= MAX_PAGES:
        print(f"\n--- Scraping Page {page_num} ---")
        listing_sel = "a.listing-card-link"
        wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, listing_sel)))
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(0.4)
        cards = driver.find_elements(By.CSS_SELECTOR, listing_sel)
        hrefs_now = [e.get_attribute('href') for e in cards if e.get_attribute('href')]
        if not hrefs_now:
            print("No URLs found on this page, stopping.")
            break
        if page_num > 1 and hrefs_now[0] == last_first_href:
            print("Listing did not change from previous page. Ending pagination.")
            break
        last_first_href = hrefs_now[0]
        new_urls = set(hrefs_now) - all_listing_urls
        print(f"Found {len(new_urls)} new listing URLs on this page.")
        all_listing_urls.update(hrefs_now)

        next_num = page_num + 1
        next_url = build_page_url(START_URL, next_num)
        prev_last = hrefs_now[-1]
        driver.get(next_url)

        t0 = time.time()
        changed = False
        while time.time() - t0 < WEBDRIVER_WAIT_TIMEOUT:
            state = driver.execute_script("return document.readyState")
            if state == "complete":
                els = driver.find_elements(By.CSS_SELECTOR, listing_sel)
                if els:
                    h = els[0].get_attribute('href')
                    hh = els[-1].get_attribute('href')
                    if h and h != last_first_href:
                        changed = True
                        break
                    if hh and hh != prev_last:
                        changed = True
                        break
            time.sleep(0.3)
        if not changed:
            print("Next page did not change within timeout. Ending pagination.")
            break

        page_num = next_num

    driver.quit()

    if all_listing_urls:
        print(f"\nTotal unique URLs collected: {len(all_listing_urls)}")
        print(f"Writing all URLs to {OUTPUT_CSV_FILE}...")
        with open(OUTPUT_CSV_FILE, 'w', newline='', encoding='utf-8') as f:
            w = csv.writer(f)
            w.writerow(['ListingURL'])
            for url in sorted(all_listing_urls):
                w.writerow([url])
        print("Process complete.")
    else:
        print("No URLs were collected.")

if __name__ == "__main__":
    main()


Initializing Chrome Driver...
Navigating to start URL: https://www.ddproperty.com/%E0%B8%A3%E0%B8%A7%E0%B8%A1%E0%B8%9B%E0%B8%A3%E0%B8%B0%E0%B8%81%E0%B8%B2%E0%B8%A8%E0%B8%82%E0%B8%B2%E0%B8%A2?listingType=sale&regionCode=TH50&locale=th&slug=search&_freetextDisplay=%E0%B9%80%E0%B8%8A%E0%B8%B5%E0%B8%A2%E0%B8%87%E0%B9%83%E0%B8%AB%E0%B8%A1%E0%B9%88&isCommercial=false

--- Scraping Page 1 ---
Found 25 new listing URLs on this page.

--- Scraping Page 2 ---
Found 20 new listing URLs on this page.

--- Scraping Page 3 ---
Found 20 new listing URLs on this page.

--- Scraping Page 4 ---
Found 20 new listing URLs on this page.

--- Scraping Page 5 ---
Found 20 new listing URLs on this page.

--- Scraping Page 6 ---
Found 20 new listing URLs on this page.

--- Scraping Page 7 ---
Found 20 new listing URLs on this page.

--- Scraping Page 8 ---
Found 20 new listing URLs on this page.

--- Scraping Page 9 ---
Found 20 new listing URLs on this page.

--- Scraping Page 10 ---
Found 20 new listing URLs

In [ ]:
#LivingInsider Scraper

import time
import csv
import re
from urllib.parse import urlparse, urlunparse, parse_qsl, urlencode
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By

START_URL = 'https://www.livinginsider.com/living_zone/45/all/all/1/%E0%B9%80%E0%B8%8A%E0%B8%B5%E0%B8%A2%E0%B8%87%E0%B9%83%E0%B8%AB%E0%B8%A1%E0%B9%88.html'
OUTPUT_CSV_FILE = 'livinginsider_listing_urls.csv'
PAGE_TIMEOUT = 45
MAX_PAGES = 200

def build_page_url(u, page):
    p = urlparse(u)
    parts = [x for x in p.path.split('/') if x]
    k = None
    for i, seg in enumerate(parts):
        if re.fullmatch(r'\d+', seg):
            k = i
    if k is None:
        base = p.path.rstrip('/')
        new_path = base if page <= 1 else base + f'/{page}'
    else:
        parts[k] = '1' if page <= 1 else str(page)
        new_path = '/' + '/'.join(parts)
    return urlunparse((p.scheme, p.netloc, new_path, p.params, urlencode(dict(parse_qsl(p.query)), doseq=True), p.fragment))

def wait_ready(driver, timeout):
    t0 = time.time()
    while time.time() - t0 < timeout:
        if driver.execute_script("return document.readyState") == "complete":
            return True
        time.sleep(0.2)
    return False

def deep_scroll(driver, rounds=18, pause=0.7):
    h0 = 0
    for _ in range(rounds):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause)
        h = driver.execute_script("return document.body.scrollHeight")
        if h == h0:
            break
        h0 = h

def collect_links_js(driver, base):
    js = """
const out = [];
const a1 = Array.from(document.querySelectorAll("a[href*='/livingdetail/'][href$='.html']"));
const a2 = Array.from(document.querySelectorAll("a[href^='/livingdetail/'][href$='.html']"));
const all = [...a1, ...a2];
for (const a of all){
  const h = a.getAttribute('href') || '';
  if (h && !h.includes('bclick') && !h.includes('stories') && !h.includes('banner')){
    out.push(h);
  }
}
return Array.from(new Set(out));
"""
    hrefs = driver.execute_script(js)
    fixed = []
    for h in hrefs:
        if h.startswith("/"):
            fixed.append(base + h)
        else:
            fixed.append(h)
    return list(dict.fromkeys(fixed))

def main():
    options = uc.ChromeOptions()
    options.add_argument('--disable-notifications')
    options.add_argument('--start-maximized')
    driver = uc.Chrome(options=options)

    base = f"{urlparse(START_URL).scheme}://{urlparse(START_URL).netloc}"

    page = 1
    all_urls = set()
    last_first = ""
    driver.get(build_page_url(START_URL, page))
    wait_ready(driver, PAGE_TIMEOUT)

    while page <= MAX_PAGES:
        deep_scroll(driver, rounds=20, pause=0.8)
        urls_now = collect_links_js(driver, base)
        if not urls_now:
            break
        if page > 1 and urls_now[0] == last_first:
            break
        last_first = urls_now[0]
        for u in urls_now:
            all_urls.add(u)

        page += 1
        prev_last = urls_now[-1]
        driver.get(build_page_url(START_URL, page))
        t0 = time.time()
        changed = False
        while time.time() - t0 < PAGE_TIMEOUT:
            wait_ready(driver, 3)
            deep_scroll(driver, rounds=6, pause=0.6)
            cur = collect_links_js(driver, base)
            if cur:
                if cur[0] != last_first or cur[-1] != prev_last:
                    changed = True
                    break
            time.sleep(0.3)
        if not changed:
            break

    driver.quit()

    if all_urls:
        with open(OUTPUT_CSV_FILE, 'w', newline='', encoding='utf-8') as f:
            w = csv.writer(f)
            w.writerow(['ListingURL'])
            for u in sorted(all_urls):
                w.writerow([u])

if __name__ == "__main__":
    main()


In [ ]:
#Kaidee Scraper

import time
import csv
import re
from urllib.parse import urlparse, urlunparse, parse_qsl, urlencode
import undetected_chromedriver as uc
from selenium.webdriver.common.by import By

START_URL = "https://baan.kaidee.com/c2p1-realestate/chiangmai"
OUTPUT_CSV_FILE = "kaidee_listing_urls.csv"
PAGE_TIMEOUT = 40
MAX_PAGES = 200

PATTERN = re.compile(r"https://baan\.kaidee\.com/product-\d+")

def build_page_url(u, page):
    if page <= 1:
        return u
    if u.endswith("/"):
        return f"{u}p-{page}"
    return f"{u}/p-{page}"

def wait_ready(driver, timeout):
    t0 = time.time()
    while time.time() - t0 < timeout:
        if driver.execute_script("return document.readyState") == "complete":
            return True
        time.sleep(0.2)
    return False

def deep_scroll(driver, rounds=12, pause=0.8):
    prev = 0
    for _ in range(rounds):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(pause)
        cur = driver.execute_script("return document.body.scrollHeight")
        if cur == prev:
            break
        prev = cur

def extract_links(html):
    urls = set(PATTERN.findall(html))
    return sorted(urls)

def main():
    options = uc.ChromeOptions()
    options.add_argument("--disable-notifications")
    options.add_argument("--start-maximized")
    driver = uc.Chrome(options=options)

    page = 1
    all_urls = set()
    last_first = ""

    while page <= MAX_PAGES:
        url = build_page_url(START_URL, page)
        driver.get(url)
        wait_ready(driver, PAGE_TIMEOUT)
        deep_scroll(driver, rounds=15, pause=0.8)

        html = driver.page_source
        links = extract_links(html)

        if not links:
            break
        if page > 1 and links[0] == last_first:
            break

        last_first = links[0]
        all_urls.update(links)
        page += 1

    driver.quit()

    if all_urls:
        with open(OUTPUT_CSV_FILE, "w", newline="", encoding="utf-8") as f:
            w = csv.writer(f)
            w.writerow(["ListingURL"])
            for u in sorted(all_urls):
                w.writerow([u])

if __name__ == "__main__":
    main()
